In [1]:
import pandas as pd
import re

# import scipy
# from scipy.stats import t
# import seaborn as sns
# from datetime import datetime, timedelta
# import plotly.express as px
# from pathlib import Path
# import statsmodels.api as sm
# from sklearn.preprocessing import StandardScaler
# from statsmodels.stats.multitest import multipletests
# from scipy.stats import f_oneway as anova
# from statsmodels.formula.api import ols 
  
# import math
# from select import select

#kernal must be restarted if these module change 
from SCP_plotter import SCP_plotter
plotter = SCP_plotter(write_output = True)
from SCP_processor import SCP_processor
processor = SCP_processor()

In [2]:
#Method put files in, one dictionary for each analysis, if there is only one file or you are looking in all files, then you don't need @'s in your SETTINGS_FILE
WRITE_OUTPUT = True
USE_MaxLFQ = False
filelist = [
             {"input1":"abundance_protein_MD.tsv",
             "input2":"abundance_peptide_MD.tsv",
             "input3":"report.pg_matrix.tsv",
             "input4":"report.pr_matrix.tsv",
             "input5":"fake.txt",
             "process_app": "FragPipe"},
             {"input1":"proteinGroups.txt",
             "input2":"peptides.txt",
             "input3":"report.pg_matrix.tsv",
             "input4":"report.pr_matrix.tsv",
             "input5":"summary.txt",
             "process_app": "MQ"},
          
            ]

            #Order of files
                #For FragPipe
                    #input1= Protein with MBR
                    #input2= Peptide with MBR
                #For DIANN
                    #input1= pg_matrix
                    #input2= pr_matrix 
                    #input3= pg_matrix MS2
                    #input4= pr_matrix MS2
                #For PD
                    #input1= Protein 
                    #input2= Peptide Groups
                    #input3= blank
                    #input4= blank
                    #input5= inpit_files


SETTINGS_FILE = "settings_test_TMT.txt" #tab delimited
    #Format
    #Required Columns
        #Group Name	
        #filter_in: a string pattern contained in desired raw filenames in analysis followed by an @ + the file number (filelist index) to look in
        #filter_out: a string pattern contained in undesired raw filenames
    #Extra columns
        #can be used to group by color, layer, or x position in CV and ID graphs

In [3]:
# define group names and assign analysis to different groups(web app version was done through GUI so
# steps are taken to make sure they have same output)
x = processor.read_files(grouped_input_files=filelist)
data_obj = processor.outer_join_data_objects(x)

# print(data_obj)
settings_table = pd.read_table(SETTINGS_FILE,sep="\t")
saved_settings = settings_table.set_index("Conditions").to_dict(orient="index")
#any run with any of the filter_out items will not be used.

# display(data_obj["protein_abundance"][pd.isna(data_obj["protein_abundance"])])
# display(data_obj["protein_abundance"])
# display(data_obj["protein_ID_Summary"])
for eachGroup in saved_settings:
    i = 0
    saved_settings[eachGroup]["records"] = []
    filterOutType = type(saved_settings[eachGroup]["filter_out"])
    if filterOutType == str or filterOutType == int or filterOutType == float and not pd.isna(saved_settings[eachGroup]["filter_out"]):
        filterOut = str.split(saved_settings[eachGroup]["filter_out"],sep = ",")
    else:
        filterOut = ["M@di"]
    if len(str.split(str(saved_settings[eachGroup]["filter_in"]),sep = "@")) > 1: #multiple files, only some have the runs for this group
        user_list = []
        #add all runs from all analyses to be probed
        for each_fileID in str.split(str(saved_settings[eachGroup]["filter_in"]),sep = "@")[1:]:
            for eachIdentifier in data_obj["run_metadata"]["Run Identifier"].drop_duplicates():    
                currentRun = data_obj["run_metadata"][data_obj["run_metadata"]["Run Identifier"] == eachIdentifier]["Run Names"] 
                if currentRun.size != 0:
                    if each_fileID == str.split(eachIdentifier,sep="-")[0] and list(currentRun)[0] not in user_list:
                        user_list.append(list(currentRun)[0])
                else:
                    print(currentRun)
        #filter for runs that relate to this gorup within those analyses
        print(user_list)
        for run_name in user_list:
            filter_in = str.split(str(saved_settings[eachGroup]["filter_in"]),sep = "@")[0]
            if bool(re.search(filter_in,run_name)) and (not any(item in run_name for item in filterOut)):
                saved_settings[eachGroup]["records"].append(list(data_obj["run_metadata"][data_obj["run_metadata"]["Run Names"] == run_name]["Run Names"])[0]) 
            else:
                print(filter_in)
                
    elif len(str.split(str(saved_settings[eachGroup]["filter_in"]),sep = "@")) == 1: #across all files or maybe there is only one
            for run_name in data_obj["run_metadata"]["Run Names"]:
                filter_in = str(saved_settings[eachGroup]["filter_in"])
                if bool(re.search(filter_in,run_name)) and (not any(item in run_name for item in filterOut)):
                    saved_settings[eachGroup]["records"].append(data_obj["run_metadata"]["Run Names"][i])  
                i = i + 1   

#add the order of each column
ignore_columns = ["filter_in","filter_out"]
category_columns = [x for x in settings_table.columns.to_list() if x not in ignore_columns]



for eachCol in category_columns:
    saved_settings["Order@"+eachCol] = settings_table[eachCol].drop_duplicates().to_list()
    

f:\Kei_PhD_Data\Greedy\SCP_TMT\SCP_processor.py:339: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prot_other_info["Source_File"] = input1
f:\Kei_PhD_Data\Greedy\SCP_TMT\SCP_processor.py:340: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pep_other_info["Source_File"] = input2
f:\Kei_PhD_Data\Greedy\SCP_TMT\SCP_processor.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

***
***
['Sachini_TKO_id_44ng_7K_25HCD_16_isolation_window_06_18_24_rep1', 'Sachini_TKO_id_44ng_7K_25HCD_16_isolation_window_06_18_24_rep2', 'Sachini_TKO_id_44ng_7K_25HCD_16_isolation_window_06_18_24_rep3']
['sample-02', 'sample-03', 'sample-04', 'sample-05', 'sample-06', 'sample-07', 'sample-08', 'sample-09', 'sample-10', 'sample-11', 'sample-02.1', 'sample-03.1', 'sample-04.1', 'sample-05.1', 'sample-06.1', 'sample-07.1', 'sample-08.1', 'sample-09.1', 'sample-10.1', 'sample-11.1', 'sample-02.2', 'sample-03.2', 'sample-04.2', 'sample-05.2', 'sample-06.2', 'sample-07.2', 'sample-08.2', 'sample-09.2', 'sample-10.2', 'sample-11.2']


In [4]:
#Missing val plot plot(protein and peptides)
plot_options={
            "mean label": "True",
            "error bar": "ci95", #None, stdev, or ci95
            "X Title": "Conditions",
            "cutoffs": [33.3,50,99],
            "Y Title": "Protein Identifications",
            "color": ["#3E6990", "#7aa8e6","#AABD8C", "#E9E3B4", "#C6878F", "#fac8d3","#F39B6D",  "pink",
                  "orange", "brown", "pink", "gray", "olive", "cyan", "black","red",
                  "yellow","green","blue","indigo","violet"],
            "width": 700,
            "height": 450,
            "font": dict(size=32, family="Arial black"),
            "ID mode": "stacked",#stacked
            "Group By X": "Conditions", #ID_Mode does MS2 vs MBR
        }
plot_options["plot_type"] = "1" # 1 is protein, 2 is peptide
figure ,_ ,_ =plotter.Missing_Values_Plots(data_obj, plot_options, saved_settings)
figure.write_image("greedy_missing_val.png", scale=10, width = 1000, height=1000)
figure.show()

['Sachini_TKO_id_44ng_7K_25HCD_16_isolation_window_06_18_24_rep1', 'Sachini_TKO_id_44ng_7K_25HCD_16_isolation_window_06_18_24_rep2', 'Sachini_TKO_id_44ng_7K_25HCD_16_isolation_window_06_18_24_rep3', 'Annotated Sequence', 'Symbol']
['sample-02', 'sample-03', 'sample-04', 'sample-05', 'sample-06', 'sample-07', 'sample-08', 'sample-09', 'sample-10', 'sample-11', 'sample-02.1', 'sample-03.1', 'sample-04.1', 'sample-05.1', 'sample-06.1', 'sample-07.1', 'sample-08.1', 'sample-09.1', 'sample-10.1', 'sample-11.1', 'sample-02.2', 'sample-03.2', 'sample-04.2', 'sample-05.2', 'sample-06.2', 'sample-07.2', 'sample-08.2', 'sample-09.2', 'sample-10.2', 'sample-11.2', 'Annotated Sequence', 'Symbol']
Index(['Symbol', '1-0-11', '1-1-11', '1-2-11'], dtype='object')
      Missing Values Rate  Symbol
0                   100.0   RPS9A
1                   100.0  RPL36B
2                     0.0    MBF1
3                     0.0    QCR6
4                     0.0    TDH2
...                   ...     ...
1112

f:\Kei_PhD_Data\Greedy\SCP_TMT\SCP_plotter.py:602: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  toPlotIDs = toPlotIDs.append(current, ignore_index = True)
f:\Kei_PhD_Data\Greedy\SCP_TMT\SCP_plotter.py:602: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  toPlotIDs = toPlotIDs.append(current, ignore_index = True)
f:\Kei_PhD_Data\Greedy\SCP_TMT\SCP_plotter.py:602: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  toPlotIDs = toPlotIDs.append(current, ignore_index = True)
f:\Kei_PhD_Data\Greedy\SCP_TMT\SCP_plotter.py:602: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  toPlotIDs = toPlotIDs.append(current, ignore_index = True)
f:\Kei_PhD_Data\Gree

In [9]:
#ID plot(protein and peptides)
plot_options={
            "mean label": "True",
            "error bar": "ci95", #None, stdev, or ci95
            "X Title": "Conditions",
            "Y Title": "Protein Identifications",
            "color": ["#3E6990", "#7aa8e6","#AABD8C", "#E9E3B4", "#C6878F", "#fac8d3","#F39B6D",  "pink",
                  "orange", "brown", "pink", "gray", "olive", "cyan", "black","red",
                  "yellow","green","blue","indigo","violet"],
            "width": 700,
            "height": 450,
            "font": dict(size=32, family="Arial black"),
            "ID mode": "total",#grouped, total, MS2, grouped_stacked, stacked
            "Group By X": "Sample Amount", #ID_Mode does MS2 vs MBR
            "Group By Color": "Processing App",#ID_Mode does MS2 vs MBR
            "Group By Stack": "ID_Mode",#ID_Mode does MS2 vs MBR
        }
plot_options["plot_type"] = "1" # 1 is protein, 2 is peptide
figure ,_ ,_ =plotter.ID_plots(data_obj, plot_options, saved_settings)
figure.write_image("asdf.png", scale=10, width = 1000, height=1000)
figure.show()

In [6]:
# CV violin plot

# print(data_obj["protein_other_info"])
plot_options={    
        "median label": "True", #only works for total and MS2, can be found in CV_summary if you set WRITE_OUTPUT = True
        "box": False,
        "X Title": "Conditions (protein)",
        "Y Title": "CV of Abundance (%)",
        "color": ["#3E6990", "#7aa8e6","#C6878F", "#fac8d3","#AABD8C", "#E9E3B4",  "#F39B6D", "#C6878F", "pink",
                  "orange", "brown", "pink", "gray", "olive", "cyan", "black","red",
                  "yellow","green","blue","indigo","violet"],
        "width": 700,
        "height": 450,
        "font": dict(size=16, family="Arial black"),
        "ylimits": [0, 500],
        "CV mode": "MS2",#grouped, total, MS2, grouped_stacked, stacked
        "Group By X": "Conditions", #ID_Mode does MS2 vs MBR
        "Group By Color": "Acquisition Mode",#ID_Mode does MS2 vs MBR
        "Group By Stack": "ID_Mode" #ID_Mode does MS2 vs MBR
    }
plot_options["plot_type"] =2  # 1 is protein, 2 is peptide
figure, _, _ =plotter.CV_plots(data_obj, plot_options, saved_settings)
figure.show()


f:\Kei_PhD_Data\Greedy\SCP_TMT\SCP_processor.py:790: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

f:\Kei_PhD_Data\Greedy\SCP_TMT\SCP_processor.py:790: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



MS2 CVs...


In [7]:
# ID Venns plot
plot_options={
            "compare groups": ["FP","MQ"],
            "title": "Venn Diagram (Protein)",
            "opacity": 0.75,
            "color": ["#3E6990", "#C6878F", "#AABD8C","#fac8d3","#7aa8e6", "#E9E3B4","#00FF00", "#FFFF00", "#FF0000", "yellow", "red",
                      "green", "purple", "orange", "brown", "pink",
                      "gray",  "olive", "cyan", "blue",  "black", ]
        }
plot_options["plot_type"] = 1 # 1 is protein, 2 is peptide
figure, _, _ =plotter.venns_plots(data_obj, plot_options, saved_settings)
figure.show()



In [8]:
# Protein Ranked Abundance plot
plot_options={
            "reference_group": "Library",
            "compare groups": ["Quant"],
            "title": "Ranked Abundance",
            "color": ["blue", "red", "green", "black", "yellow", "purple",
                      "orange", "brown", "pink", "gray", "olive", "cyan"],
            "symbol": ['circle', 'circle', 'circle'],
            "marker_size": 8,
            "width": 700,
            "height": 450,
            "font": dict(size=16, family="Arial black"),
            "ylimits": [],
            "xlimits": [],
            "X Title": "Ranked Abundance",
            "Y Title": "Average Reporter Ion Abundance",
        }
figure, _, _ =plotter.Rank_Abundance_Plots(data_obj, plot_options, saved_settings)
figure.show()

KeyError: 'Library'

In [ ]:
# Protein Abundance Volcano plot
plot_options={
            "compare groups": ["MQ","FP"],
            "title": "Volcano Plot",
            "X Title": "Log2 Fold Change",
            "Y Title": "-Log10(adjusted p-value)",
            "up color": "red",
            "down color": "blue",
            "all color": "gray",
            "width": 700,
            "height": 450,
            "font": dict(size=16, family="Arial black"),
            "ylimits": [],
            "xlimits": []
        }
figure, _, _ =plotter.volcano_plots(data_obj, plot_options, saved_settings)
figure.show()


In [ ]:
# Protein Abundance PCA plot
plot_options={
            "compare groups": ["MQ","FP"],
            "title": "PCA Analysis",
            "color": ["blue", "red", "green", "black", "yellow", "purple",
                      "orange", "brown", "pink", "gray", "olive", "cyan"],
            "symbol": ['circle', 'circle', 'circle'],
            "marker_size": 8,
            "width": 700,
            "height": 450,
            "font": dict(size=16, family="Arial black"),
            "ylimits": [],
            "xlimits": []
        }
figure, _, _ =plotter.PCA_plots(data_obj, plot_options, saved_settings)
figure.show()


In [ ]:
# Protein Abundance Heat Map
plot_options={
            "compare groups": ["Quant","Library"],
            "log2_transform": False,
            "significant_only": True,
            "alpha": 0.05, 
            "min_fold_change": 2
        }
figure, _, _ =plotter.heatmap_plots(data_obj, plot_options, saved_settings)
figure.show()

In [ ]:
# ID Venns plot
plot_options={
            "compare groups": ["Quant","Library"],
            "inclusion list": "proteins_symbols.csv",
            "title": "Venn Diagram (Protein)",
            "opacity": 0.75,
            "color": ["#3E6990", "#C6878F", "#AABD8C","#fac8d3","#7aa8e6", "#E9E3B4","#00FF00", "#FFFF00", "#FF0000", "yellow", "red",
                      "green", "purple", "orange", "brown", "pink",
                      "gray",  "olive", "cyan", "blue",  "black", ]
        }
plot_options["plot_type"] = 1 # 1 is protein, 2 is peptide
figure, _, _ =plotter.inclusion_venn_plots(data_obj, plot_options, saved_settings)
figure.show()

